## Time Series Forecasting
1. Naive method, Simple average method and the simple moving average method
2. Exponential Smoothing: Simple Exponential Smoothing technique, Holt’s method with trend and Holt Winter’s method
3. Auto Regressive methods: ARIMA/SARIMA
4. Machine Learning methods: CNN x LSTM
5. Hybrids: Prophet

In [ ]:
import numpy as np
import pandas as pd
import pmdarima as pm
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import math
from scipy.stats import variation

%matplotlib  inline
warnings.filterwarnings('ignore')
sns.set_style("darkgrid")

import sklearn
from sklearn.metrics import mean_squared_error

In [ ]:
df = pd.read_csv(r'C:\Users\a844050\Documents\Github\personal\data_science\forecasting\time_series\datasets\coffee_sales.csv')

In [ ]:
print(f"{df.shape[0]} Rows & {df.shape[1]} Columns")
print("------")
print(df.dtypes)
df.head(5)

### Auto Regressive Methods

#### >> Stationary Test

In [ ]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

def results_adfuller_test(pvalue, sig_level):
    if pvalue < sig_level: 
        return print(">> REJECT H_0: Time series is stationary, proceed without differencing")
    return print(">> Cannot reject H_0: Time series is not stationary, differencing is required")

result = adfuller(df['money'], autolag='AIC')
print(f'ADF Statistic: {result[0]}')
print(f'p-value: {result[1]}')
for key, value in result[4].items():
    print('Critial Values:')
    print(f'   {key}, {value}') 

print("\n")
results_adfuller_test(result[1],0.05)

#### >> Differencing

In [ ]:
df_diff = df['money'].diff().dropna()
result = adfuller(df_diff, autolag='AIC')

print(f'ADF Statistic: {result[0]}')
print(f'p-value: {result[1]}')
for key, value in result[4].items():
    print('Critial Values:')
    print(f'   {key}, {value}') 

print("\n")
results_adfuller_test(result[1],0.05)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

plot_acf(df_diff, ax=axes[0])
plot_pacf(df_diff, ax=axes[1])

axes[0].set_title('ACF Plot')
axes[1].set_title('PACF Plot')

plt.tight_layout()
plt.show()

#### Time Series Decomposition

In [ ]:
import matplotlib.dates as mdates
from statsmodels.tsa.seasonal import seasonal_decompose

temp = df.reset_index()[['datetime', 'money']].copy()
temp.index = np.array(temp['datetime'], dtype=np.datetime64)
temp = temp[['money']]

print('Daily Coffee Sales (Sum) Decomposition')
daily_data = temp.resample('D').sum()
result = seasonal_decompose(daily_data.dropna(), model='additive')
result.plot()
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%d'))
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.api import ExponentialSmoothing as ExponentialSmoothingV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from pmdarima import auto_arima
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt

# Import timeseries data
data = daily_data

# Split data into training and test sets
train = data[:int(len(data)*0.8)]
test = data[int(len(data)*0.8):]

# Prepare the data for CNN-LSTM
def create_dataset(series, look_back=7):
    X, y = [], []
    for i in range(len(series) - look_back):
        X.append(series[i:i + look_back])
        y.append(series[i + look_back])
    return np.array(X), np.array(y)

look_back = 7  # Number of time steps to look back
X_train, y_train = create_dataset(train.values, look_back)
X_test, y_test = create_dataset(test.values, look_back)

# Normalize data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape for CNN-LSTM (samples, timesteps, features)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Build CNN-LSTM model
cnn_lstm_model = Sequential()
cnn_lstm_model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(look_back, 1)))
cnn_lstm_model.add(MaxPooling1D(pool_size=2))
cnn_lstm_model.add(Dropout(0.2))
cnn_lstm_model.add(LSTM(50, return_sequences=True))
cnn_lstm_model.add(LSTM(50))
cnn_lstm_model.add(Dense(1))

cnn_lstm_model.compile(optimizer='adam', loss='mean_squared_error')

# Train the CNN-LSTM model
cnn_lstm_history = cnn_lstm_model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1, verbose=1)

# Predict using CNN-LSTM
cnn_lstm_y_pred = cnn_lstm_model.predict(X_test)

# 1. Naive Method, Simple Average, and Simple Moving Average
naive_forecast = pd.Series([train.iloc[-1]] * len(test), index=test.index)
simple_average_forecast = pd.Series([np.mean(train)] * len(test), index=test.index)
moving_average = train.rolling(window=7).mean().iloc[-1]
moving_average_forecast = pd.Series([moving_average] * len(test), index=test.index)

# 2. Exponential Smoothing Methods
# Simple Exponential Smoothing
ses_model = ExponentialSmoothing(train, trend=None, seasonal=None, seasonal_periods=None)
ses_fit = ses_model.fit()
ses_forecast = ses_fit.predict(start=test.index[0], end=test.index[-1])

# Holt’s Method (with Trend)
holt_model = ExponentialSmoothingV2(train, trend='add', seasonal=None, seasonal_periods=None)
holt_fit = holt_model.fit()
holt_forecast = holt_fit.predict(start=test.index[0], end=test.index[-1])

# Holt-Winters Method (with Trend and Seasonality)
hw_model = ExponentialSmoothingV2(train, trend='add', seasonal='add', seasonal_periods=7)  # Weekly seasonality
hw_fit = hw_model.fit()
hw_forecast = hw_fit.predict(start=test.index[0], end=test.index[-1])

# 3. pmdarima auto_arima for optimal ARIMA/SARIMA
auto_arima_model = auto_arima(train, seasonal=True, m=7,d=None,stepwise=True, trace=True)
auto_arima_forecast = auto_arima_model.predict(n_periods=len(test))

# 4. CNN-LSTM Forecast
cnn_lstm_y_pred_flat = cnn_lstm_y_pred.flatten()

# Evaluation Metrics
def evaluate_forecast(forecast, actual):
    mae = mean_absolute_error(actual, forecast)
    mse = mean_squared_error(actual, forecast)
    rmse = sqrt(mse)
    return mae, mse, rmse

naive_mae, naive_mse, naive_rmse = evaluate_forecast(naive_forecast, test)
simple_average_mae, simple_average_mse, simple_average_rmse = evaluate_forecast(simple_average_forecast, test)
moving_average_mae, moving_average_mse, moving_average_rmse = evaluate_forecast(moving_average_forecast, test)
ses_mae, ses_mse, ses_rmse = evaluate_forecast(ses_forecast, test)
holt_mae, holt_mse, holt_rmse = evaluate_forecast(holt_forecast, test)
hw_mae, hw_mse, hw_rmse = evaluate_forecast(hw_forecast, test)
auto_arima_mae, auto_arima_mse, auto_arima_rmse = evaluate_forecast(auto_arima_forecast, test)

# Plotting
plt.figure(figsize=(14, 10))
plt.plot(data, label='Original Data')
plt.plot(test.index, naive_forecast, label='Naive Forecast', color='yellow')
plt.plot(test.index, simple_average_forecast, label='Simple Avergae', color='cyan')
plt.plot(test.index, moving_average_forecast, label='Moving Average', color='purple')
plt.plot(test.index, ses_forecast, label='Simple Exponential Smoothing', color='orange')
plt.plot(test.index, holt_forecast, label='Holt’s Method', color='green')
plt.plot(test.index, hw_forecast, label='Holt-Winters Method', color='red')
plt.plot(test.index, auto_arima_forecast, label='pmdarima auto_arima', color='blue')
plt.plot(test.index[look_back:], cnn_lstm_y_pred_flat, label='CNN-LSTM', color='cyan')
plt.axvline(x=train.index[-1], color='grey', linestyle='--', label='Train/Test Split')
plt.legend()
plt.title('Time Series Forecasting')
plt.show()

In [ ]:
print("Evaluation Metrics:")
print(f"Naive Method: MAE={naive_mae:.3f}, MSE={naive_mse:.3f}, RMSE={naive_rmse:.3f}")
print(f"Simple Average: MAE={simple_average_mae:.3f}, MSE={simple_average_mse:.3f}, RMSE={simple_average_rmse:.3f}")
print(f"Moving Average: MAE={moving_average_mae:.3f}, MSE={moving_average_mse:.3f}, RMSE={moving_average_rmse:.3f}")
print(f"Simple Exponential Smoothing: MAE={ses_mae:.3f}, MSE={ses_mse:.3f}, RMSE={ses_rmse:.3f}")
print(f"Holt’s Method: MAE={holt_mae:.3f}, MSE={holt_mse:.3f}, RMSE={holt_rmse:.3f}")
print(f"Holt-Winters Method: MAE={hw_mae:.3f}, MSE={hw_mse:.3f}, RMSE={hw_rmse:.3f}")
print(f"pmdarima auto_arima: MAE={auto_arima_mae:.3f}, MSE={auto_arima_mse:.3f}, RMSE={auto_arima_rmse:.3f}")
print(f"Chosen SARIMA model: {auto_arima_model}")
print(f"CNN-LSTM: MAE={cnn_lstm_mae:.3f}, MSE={cnn_lstm_mse:.3f}, RMSE={cnn_lstm_rmse:.3f}")

